In [1]:
!nvcc -V
!echo
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0

gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
!git clone https://github.com/SwinTransformer/Swin-Transformer-Object-Detection.git
!pip install -r /content/Swin-Transformer-Object-Detection/requirements.txt
!pip install openmim
!mim install mmdet

In [ ]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd ..

In [4]:
%cd Swin-Transformer-Object-Detection/

/content/Swin-Transformer-Object-Detection


In [ ]:
!unzip '/content/drive/MyDrive/input/ocr_ancient.zip' -d /content/data

In [7]:
# 목판본_043_언해류_대학언해_01
# 목판본_043_언해류_대학언해1_01
# 필사본_104_문학류_대명영렬전2_01
# 필사본_104_문학류_대병영렬전2_01
# 필사본_105_문학류_대명영렬전3_01
# 필사본_105_문학류_대병영렬전3_01
import os
from glob import glob
json_list = glob(os.path.join('/content/data/이미지데이터','*')) # derectory
for i in json_list:
    tmp_list = glob(os.path.join(i,'*'))
    for j in tmp_list: # image파일

        if j.split('/')[-1][:-8] != i.split('/')[-1]:
            dir_name = i.split('/')[-1]
            img_number = j.split('/')[-1][-8:]
            os.rename(j, '/content/data/이미지데이터/'+dir_name+'/'+dir_name+img_number)

In [8]:
import sys
from mmcv import Config
from mmcv.runner import load_checkpoint
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector, set_random_seed, init_detector
from mmdet.datasets import build_dataloader, build_dataset, replace_ImageToTensor
from mmdet.utils import collect_env, get_root_logger

import torch
import time
import json

from pprint import pprint
import os

In [11]:
classes = (
    "word",
)
CUR_PATH = os.getcwd()
CFG_PATH = "/content/drive/Othercomputers/내 컴퓨터/workspace/ocr_ancient_korean/detection/configs/cascade_rcnn/cascade_rcnn_swin_tiny_fpn_1x_coco.py"
PREFIX = "/content/data/이미지데이터"
WORK_DIR = "/content/drive/MyDrive/Colab Notebooks/Computer_Vision/work_dir"
# CHK_PATH = "/content/drive/MyDrive/Colab Notebooks/lesion_obd/cascade_mask_rcnn_swin_small_patch4_window7.pth"
# config file 들고오기
cfg = Config.fromfile(CFG_PATH)

In [12]:
for i in cfg:
    print(i, '*'*40)
    print(cfg[i])

model ****************************************
{'type': 'CascadeRCNN', 'pretrained': None, 'backbone': {'type': 'SwinTransformer', 'embed_dim': 96, 'depths': [2, 2, 6, 2], 'num_heads': [3, 6, 12, 24], 'window_size': 7, 'mlp_ratio': 4.0, 'qkv_bias': True, 'qk_scale': None, 'drop_rate': 0.0, 'attn_drop_rate': 0.0, 'drop_path_rate': 0.2, 'ape': False, 'patch_norm': True, 'out_indices': (0, 1, 2, 3), 'use_checkpoint': False}, 'neck': {'type': 'FPN', 'in_channels': [96, 192, 384, 768], 'out_channels': 256, 'num_outs': 5}, 'rpn_head': {'type': 'RPNHead', 'in_channels': 256, 'feat_channels': 256, 'anchor_generator': {'type': 'AnchorGenerator', 'scales': [8], 'ratios': [0.7, 1.0, 1.3], 'strides': [4, 8, 16, 32, 64]}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [1.0, 1.0, 1.0, 1.0]}, 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0}, 'loss_bbox': {'type': 'SmoothL1Loss', 'beta': 0.1111111111111111, 'loss_weigh

In [13]:

cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = '/content/drive/MyDrive/input/train.json'

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = '/content/drive/MyDrive/input/valid.json'

cfg.data.samples_per_gpu = 4
cfg.data.workers_per_gpu = 4

cfg.seed = 42
cfg.gpu_ids = [0]

distributed = False

cfg.work_dir = WORK_DIR
cfg.runner.max_epochs = 20
cfg.rtotal_epochs = 20
cfg.optimizer = dict(type='Adam', lr=0.0001, weight_decay=0.0001)

cfg.lr_config = dict(
    policy='CosineAnnealing', # The policy of scheduler, also support CosineAnnealing, Cyclic, etc. Refer to details of supported LrUpdater from https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/lr_updater.py#L9.
    by_epoch=False,
    warmup='linear', # The warmup policy, also support `exp` and `constant`.
    warmup_iters=500, # The number of iterations for warmup
    warmup_ratio=0.001, # The ratio of the starting learning rate used for warmup
    min_lr=1e-07)

cfg.log_config.interval = 600
cfg.checkpoint_config.interval = 1
cfg.log_config = {'hooks': [{'type': 'TextLoggerHook'}], 'interval': 600}
# cfg.model.pretrained = None

model = build_detector(cfg.model)

datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES

loading annotations into memory...
Done (t=24.44s)
creating index...
index created!


In [14]:
train_detector(model, datasets[0], cfg, distributed=distributed, validate=True)

loading annotations into memory...
Done (t=2.72s)
creating index...


2021-12-08 00:05:52,428 - mmdet - INFO - load checkpoint from local path: /content/drive/Othercomputers/내 컴퓨터/workspace/ocr_ancient_korean/detection/pth/swin_tiny_patch4_window7_224.pth


index created!


2021-12-08 00:05:53,970 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: model

missing keys in source state_dict: backbone.patch_embed.proj.weight, backbone.patch_embed.proj.bias, backbone.patch_embed.norm.weight, backbone.patch_embed.norm.bias, backbone.layers.0.blocks.0.norm1.weight, backbone.layers.0.blocks.0.norm1.bias, backbone.layers.0.blocks.0.attn.relative_position_bias_table, backbone.layers.0.blocks.0.attn.relative_position_index, backbone.layers.0.blocks.0.attn.qkv.weight, backbone.layers.0.blocks.0.attn.qkv.bias, backbone.layers.0.blocks.0.attn.proj.weight, backbone.layers.0.blocks.0.attn.proj.bias, backbone.layers.0.blocks.0.norm2.weight, backbone.layers.0.blocks.0.norm2.bias, backbone.layers.0.blocks.0.mlp.fc1.weight, backbone.layers.0.blocks.0.mlp.fc1.bias, backbone.layers.0.blocks.0.mlp.fc2.weight, backbone.layers.0.blocks.0.mlp.fc2.bias, backbone.layers.0.blocks.1.norm1.weight, backbone.layers.0.blocks.1.no

KeyError: ignored